## The HealthBot Prototype
Your task is to create a LangGraph-based workflow for the HealthBot that includes the following functionality:

Ask the patient what health topic or medical condition they'd like to learn about.  
Use the Tavily search engine (via LangChain community tool) to find relevant, up-to-date medical information on the topic.  
Summarize the Tavily search results into a patient-friendly explanation.  
Present the summarized information to the patient and allow them to read it.  
Prompt the patient to indicate when they're ready for a comprehension check.  
Generate a single, relevant quiz question based on the provided information.  
Present the quiz question to the patient.  
Allow the patient to enter their answer to the quiz question.  
Evaluate the patient's response, providing a grade and explanation. The explanation should include relevant citations from the summary to reinforce learning.  
Present the grade and explanation to the patient.  
Ask if the patient would like to learn about another health topic or exit the session.  
Either restart the flow for a new topic or end the session. Ensure that the state is reset when starting a new topic to maintain privacy and accuracy.

In [43]:
import os
from typing import Any, Dict, List, Literal, TypedDict, cast

# LangChain/LangGraph imports
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph

## from langgraph.graph.message import add_messages

# --- 1. SETUP AND CONFIGURATION ---
# IMPORTANT: Ensure these environment variables are set before running:
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["TAVILY_API_KEY"] = "..."

# Initialize LLM and Tool
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
tavily_tool = TavilySearchResults(max_results=3)



In [44]:
# --- 2. STATE DEFINITION (With total=False) ---
class HealthBotState(TypedDict, total=False):
    """
    Represents the state of our HealthBot conversation.
    total=False makes all fields OPTIONAL, requiring the use of .get()
    for safe access in node functions.
    """
    topic: str
    search_results: List[Dict[str, Any]]
    summary: str
    quiz_question: str
    quiz_answer: str
    grade_explanation: str
    flow_status: Literal[None, "ASK_TOPIC", "READY_FOR_QUIZ", "GRADED"]

In [45]:
# --- 3. GRAPH NODE FUNCTIONS (The Logic) ---

def start_session(state: HealthBotState) -> HealthBotState:
    """
    Initial prompt to the user and captures the health topic.
    Handles flow restart by checking the flow_status.
    """
    if state.get("topic"):
        print("\n" + "="*50)
        print("🤖 HealthBot: Starting a New Session...")
        print("="*50)
        # Return minimal state update to clear data and prompt for new topic
        return {"topic": "", "flow_status": None, "search_results": [], "summary": "", "quiz_question": "", "quiz_answer": "", "grade_explanation": ""}

    # Initial run logic
    print("\n" + "="*50)
    print("Welcome to HealthBot: Your AI-Powered Patient Education System!")
    print("="*50)

    topic = input("🤖 HealthBot: What medical condition or health topic would you like to learn about today? ")

    print(f"\n🧠 HealthBot: Searching for up-to-date information on **{topic}**...")

    return {"topic": topic}


def retrieve_and_summarize(state: HealthBotState) -> HealthBotState:
    """
    Uses Tavily to search for information and the LLM to summarize it.
    """
    topic = state.get("topic")

    if not topic:
        print("❌ Error: Topic not found in state. Returning to start.")
        return {"flow_status": "ASK_TOPIC"}

    search_results = tavily_tool.invoke({"query": f"{topic} patient education AND treatment AND symptoms"})

    search_context = "\n---\n".join([f"Source {i+1}:\n{res['content']}" for i, res in enumerate(search_results)])

    # 3. Create the patient-friendly summary prompt
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", (
            "You are a compassionate, expert healthcare educator. Your task is to summarize the provided "
            "medical context into a simple, patient-friendly explanation aimed at a general audience "
            "(6th-grade reading level). The summary must be clear, concise, and cover the condition, "
            "main symptoms, and basic treatment options. The tone should be supportive and encouraging."
            "\n\nCONTEXT:\n{context}"
        )),
        ("human", "Please summarize the information about '{topic}' for a patient.")
    ])

    # 4. Invoke the LLM for summarization
    # Correctly build the chain using 'summary_prompt'
    summary_chain = summary_prompt | llm

    # FIX: Explicitly cast the output to str to satisfy the type checker.
    summary = cast(str, summary_chain.invoke({"context": search_context, "topic": topic}).content)

    print("✅ HealthBot: Information summarized and ready.")

    return {"search_results": search_results, "summary": summary}


def present_and_prompt_quiz(state: HealthBotState) -> HealthBotState:
    """
    Presents the summary to the patient and asks if they are ready for the quiz.
    """
    topic = state.get("topic", "the condition")
    summary = state.get("summary")

    if not summary:
        print("❌ Error: Summary not generated. Returning to start.")
        return {"flow_status": "ASK_TOPIC"}

    print("\n" + "—"*50)
    print(f"📄 **Your Learning Summary: {topic.upper()}**")
    print("—"*50)
    print(summary)
    print("—"*50)

    while True:
        ready = input("🤖 HealthBot: Please read the summary above. When you are ready for a quick comprehension check (quiz) type 'yes': ").strip().lower()
        if ready == 'yes':
            print("\n📝 HealthBot: Great! Generating your quiz question now...")
            break
        else:
            print("❌ Invalid input. Please type 'yes' when ready.")

    return {"flow_status": "READY_FOR_QUIZ"}


def generate_and_ask_quiz(state: HealthBotState) -> HealthBotState:
    """
    Generates a single quiz question and captures the patient's answer.
    """
    summary = state.get("summary")

    if not summary:
        print("❌ Error: Summary is missing for quiz generation. Returning to start.")
        return {"flow_status": "ASK_TOPIC"}

    # 1. Quiz Generation Prompt
    quiz_prompt = ChatPromptTemplate.from_messages([
        ("system", (
            "Based ONLY on the following summary, generate a single, relevant, open-ended "
            "quiz question to check the patient's comprehension. Do not provide the answer."
            "\n\nSUMMARY:\n{summary}"
        )),
        ("human", "Generate one comprehension quiz question.")
    ])

    # 2. FIX: Correctly build the chain using the new 'quiz_prompt'
    quiz_chain = quiz_prompt | llm

    # Invoke LLM to generate the question
    # The input to the invoke must match the placeholder in the prompt ({summary})
    quiz_question = cast(str, quiz_chain.invoke({"summary": summary}).content) # Cast LLM output to str

    # 3. Present question and capture answer
    print("\n" + "="*50)
    print("🧠 Comprehension Check")
    print("="*50)
    print(f"❓ Question: **{quiz_question}**")
    user_answer = input("👤 Your Answer: ")

    return {"quiz_question": quiz_question, "quiz_answer": user_answer}


def evaluate_response(state: HealthBotState) -> HealthBotState:
    """
    Evaluates the patient's response and generates a reinforcing explanation.
    """
    summary = state.get("summary")
    question = state.get("quiz_question")
    user_answer = state.get("quiz_answer")

    if not all([summary, question, user_answer]):
        print("❌ Error: Missing quiz data for evaluation. Returning to start.")
        return {"flow_status": "ASK_TOPIC"}

    print("\n⏳ HealthBot: Evaluating your response...")

    evaluation_prompt = ChatPromptTemplate.from_messages([
        ("system", (
            "You are grading a patient's quiz answer. You must do three things in your response:\n"
            "1. **Grade:** Start your response with either 'CORRECT', 'PARTIALLY CORRECT', or 'INCORRECT' (in all caps).\n"
            "2. **Explanation:** Provide a supportive, gentle explanation of why the answer received that grade.\n"
            "3. **Reinforcement:** Cite and directly quote the relevant sentence(s) from the SUMMARY to reinforce the learning."
            "\n\nSUMMARY:\n{summary}"
        )),
        ("human", (
            "Question: {question}\n"
            "Patient's Answer: {answer}"
        ))
    ])

    evaluation_chain = evaluation_prompt | llm
    grade_explanation = cast(str, evaluation_chain.invoke({ # Cast LLM output to str
        "summary": summary,
        "question": question,
        "answer": user_answer
    }).content)

    return {"grade_explanation": grade_explanation, "flow_status": "GRADED"}


def display_grade_and_check_exit(state: HealthBotState) -> Dict[str, str]:
    """
    Displays the final grade and asks the user if they want to exit or restart.
    Returns 'continue' (for new topic) or 'end' (to exit).
    """
    grade_explanation = state.get("grade_explanation", "No grade explanation was generated.")

    print("\n" + "="*50)
    print("✨ Quiz Results and Explanation")
    print("="*50)
    print(grade_explanation)
    print("="*50)

    while True:
        choice = input("🤖 HealthBot: Would you like to learn about another health topic? (Type 'yes' to start new, or 'no' to exit): ").strip().lower()
        if choice == 'yes':
            return {"next_step": "continue"}
        elif choice == 'no':
            return {"next_step": "end"}
        else:
            print("❌ Invalid input. Please type 'yes' or 'no'.")


In [46]:
# --- 4. BUILD THE LANGGRAPH WORKFLOW ---

workflow = StateGraph(HealthBotState)

workflow.add_node("start_session", start_session)
workflow.add_node("retrieve_and_summarize", retrieve_and_summarize)
workflow.add_node("present_and_prompt_quiz", present_and_prompt_quiz)
workflow.add_node("generate_and_ask_quiz", generate_and_ask_quiz)
workflow.add_node("evaluate_response", evaluate_response)
workflow.add_node("display_grade_and_check_exit", display_grade_and_check_exit)

workflow.set_entry_point("start_session")

workflow.add_edge("start_session", "retrieve_and_summarize")
workflow.add_edge("retrieve_and_summarize", "present_and_prompt_quiz")
workflow.add_edge("present_and_prompt_quiz", "generate_and_ask_quiz")
workflow.add_edge("generate_and_ask_quiz", "evaluate_response")
workflow.add_edge("evaluate_response", "display_grade_and_check_exit")

workflow.add_conditional_edges(
    "display_grade_and_check_exit",
    lambda state: state["next_step"],
    {
        "continue": "start_session",
        "end": END
    }
)

app = workflow.compile()

In [47]:
# --- 5. EXECUTION ---

if __name__ == "__main__":

    if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
        print("ERROR: Please set OPENAI_API_KEY and TAVILY_API_KEY environment variables.")
    else:
        print("🚀 HealthBot Initialized. Starting conversation...")

        initial_state: HealthBotState = {}

        try:
            app.invoke(initial_state)

            print("\n👋 HealthBot: Thank you for using the patient education system. Goodbye!")

        except Exception as e:
            print(f"\nAn error occurred during execution: {e}")
            print("\nCheck your code, API keys, or input to ensure the required data is present at each step.")

🚀 HealthBot Initialized. Starting conversation...

Welcome to HealthBot: Your AI-Powered Patient Education System!

🧠 HealthBot: Searching for up-to-date information on **scoliosis**...
✅ HealthBot: Information summarized and ready.

——————————————————————————————————————————————————
📄 **Your Learning Summary: SCOLIOSIS**
——————————————————————————————————————————————————
Scoliosis is a condition where the spine curves from side to side, instead of being straight. This can cause changes in posture, like uneven shoulders or hips, and sometimes back pain. You might notice one shoulder blade sticking out more or one side of the rib cage pushing forward.

Most of the time, scoliosis is mild and doesn't cause serious problems. Doctors usually find it during the teenage years. Treatment can include physical therapy, wearing a brace, or sometimes surgery. The goal is to help with any pain and keep the curve from getting worse.

Remember, many people with scoliosis live normal, active lives. I